In [1]:

import mleap.pyspark
from mleap.pyspark.spark_support import SimpleSparkSerializer

In [2]:

# This notebook creates and exports a sklearn pipeline to MLeap for deployment into production

# MLeap allows data scientists and engineers to deploy machine learning pipelines from Spark and 
# Scikit-Learn to a portable format and execution engine.

import pandas as pd

from mleap.sklearn.pipeline import Pipeline
from mleap.sklearn.preprocessing.data import FeatureExtractor, LabelEncoder, ReshapeArrayToN1
from sklearn.preprocessing import OneHotEncoder

In [3]:

data = pd.DataFrame(['a', 'b', 'c'], columns=['col_a'])

categorical_features = ['col_a']

feature_extractor_tf = FeatureExtractor(input_scalars=categorical_features, 
                                         output_vector='imputed_features', 
                                         output_vector_items=categorical_features)

In [4]:

# Label Encoder for x1 Label 
label_encoder_tf = LabelEncoder(input_features=feature_extractor_tf.output_vector_items,
                               output_features='{}_label_le'.format(categorical_features[0]))

# Reshape the output of the LabelEncoder to N-by-1 array
reshape_le_tf = ReshapeArrayToN1()


In [5]:

# Vector Assembler for x1 One Hot Encoder
one_hot_encoder_tf = OneHotEncoder(sparse=False)

# WORKAROUND: changed line 90 in offending 'preprocessing/data.py' file
# from: class_name = "{}".format(self.init.im_class).split('.')[-1].replace('>','').replace("'",'')
# to: class_name = "{}".format(self.__init__.__class__.__name__).split('.')[-1].replace('>','').replace("'",'')
one_hot_encoder_tf.mlinit(prior_tf = label_encoder_tf, 
                          output_features = '{}_label_one_hot_encoded'.format(categorical_features[0]))

one_hot_encoder_pipeline_x0 = Pipeline([
                                         (feature_extractor_tf.name, feature_extractor_tf),
                                         (label_encoder_tf.name, label_encoder_tf),
                                         (reshape_le_tf.name, reshape_le_tf),
                                         (one_hot_encoder_tf.name, one_hot_encoder_tf)
                                        ])

In [6]:

one_hot_encoder_pipeline_x0.mlinit()
one_hot_encoder_pipeline_x0.fit_transform(data)
one_hot_encoder_pipeline_x0.serialize_to_bundle('/tmp', 'mleap-scikit-test-pipeline', init=True)


/home/nm16667/anaconda3/lib/python3.6/site-packages/mleap/sklearn/preprocessing/data.py:390: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(X, warn=True)
